# Architecture Guidelines for GeoAdmin Infrastructure

## Micro-Service oriented architecture
- Self contained: independently deployable artifact with high functional cohesion ("functionalities belong together") and as little dependencies to other projects and resources as possible. 
- Micro-Service manages it's data
- Primary access to data is via API, documentation is openapi

Furthermore, a service has

## CI/CD
Each service or productive piece of software should be automatically tested, before merged to `develop`. AWS code pipeline is used as managed CI environment, consisting of `code build`, `code test` and `code deploy`. During `code build`, the [unit tests](#unit-tests) are performed and if successful, the software is packaged and a versioned artefact is generated. Packaging is usually a Docker container further described in [packaging](#packaging).

## Packaging
Docker is used as packaging method for all projects. A project must provide a build command that builds one or more docker images with AWS code build. AWS container registry must be used to store the built containers.
Base images should be chosen as suggested in https://pythonspeed.com/articles/base-image-python-docker-images/. Currently this are buster-based images, python:3.X-slim-buster, debian:buster or ubuntu:18.04 (to be replaced by ubuntu:20.04). An excellent guide to Docker packaging can be found here: https://pythonspeed.com/docker/.
Development should happen locally with framework-provided runtime (e.g. "./manage.py runserver" for django, "env FLASK_APP=hello.py flask run" for flask, "npm run server" for webpack, etc.). 

## Runtime
Kubernetes is used as container orchestration platform and should be the main runtime environment to run the Docker images (12-factor apps). In cases where orchestration is not necessary or doensn't make sense (e.g. stateful apps), containers can be started via `systemd`.

## Deploy
- script python
- bash
- deploy-package von c2c
- Makefile

## Logs
Each service must write logs that are captured in a centralized ELK infrastructure. 

## Testing
We distinguish two categories of tests, unit tests and integration test. 

### Unit Tests
Unit Tests can be performed before a docker image is built using a dedicated test runner for Unit tests. Unit Tests furthermore don't require external resources. If useful, external resources can be mocked in Unit Tests. 

### Integration Tests
Integration Tests are performed with the built docker image and have access to external resources.

## APIs
Services should structure their api in the following way:

`xyz.bgdi.ch/api/v3/`
`xyz.bgdi.ch/doc/v3/`

REST APIs should respect the following best practices:

Documentation is provided in [openapi](https://swagger.io/docs/specification/about/) standard

# Python
Readability over all!

#### Fail-Fast
- When there is missing environment variable or start-up parameters, instead of still starting up the system normally or using fall-back strategy (fall-back to default environments/parameters), the system should fail and stop so that we can be notified and fix the problem right away.

- When a client sends a request with invalid parameters, instead of silently correct the parameters and continue handling the request, the server should let the request fail so that client can be notified and fix the problem as soon as possible.

- Exceptions should never be silently swallowed. Exceptions should only be caught when the catcher know how to handle it; otherwise, let the exception be thrown outside. And let the app crash if no part of the app knows how to handle it (An exception caused by unexpected bugs).

#### Introduce Explaining Variable
This will help to explain the meaning of each variable when expressions are hard to read.
```python
# maybe not the most illustrative example, but you get the idea
# change
if ( "MAC" in platform.upper() and \
    "IE" in browser.upper() and \
    wasInitialized() and \
    resize > 0 ):
    # do something

# to
isMacOs = "MAC" in platform.upper()
isIEBrowser = "IE" in browser.upper()
wasResized = resize > 0
if (isMacOs and isIEBrowser and wasInitialized() and wasResized):
    # do something
```